In [48]:
import os
import sys
import pandas as pd
import utils as utils
import datetime
import datetime as dt
from pathlib import Path



# Simplifying Tables

Takes all values from a specific date onwards. Values on all tables identified as of Booking date in Appointments table accordingly.

In [ ]:
# CONFIG

date_cut = dt.datetime(2025, 1, 1)  # CHANGE AS DESIRED

SRC_DIR = Path("../data/gcc/")
SAVE_DIR = Path("../data/gcc_simplified") / date_cut.strftime("%d_%m_%Y")
SAVE_DIR.mkdir(parents=True, exist_ok=True)



# --- Load ---
ORIGINAL_dfs = utils.GetSavedTables(str(SRC_DIR))

def save_pickle(df: pd.DataFrame, table_name: str) -> Path:
    out_path = SAVE_DIR / f"{table_name}.pkl"
    df.to_pickle(out_path)
    return out_path

def require_cols(df: pd.DataFrame, table: str, cols: list) -> None:
    missing = [c for c in cols if c not in df.columns]
    if missing:
        raise KeyError(f"{table} is missing columns: {missing}. Available: {list(df.columns)}")

# --- 1) Filter Appointment by BookedDate ---
appt_table = "ML_DataModel.Appointment"
df_appt = ORIGINAL_dfs[appt_table].copy()

require_cols(df_appt, appt_table, ["BookedDate", "Episode", "Patient"])

df_appt["BookedDate"] = pd.to_datetime(df_appt["BookedDate"], errors="coerce")

# If BookedDate has timezones, align date_cut to timezone-aware, otherwise compare naive-naive.
if getattr(df_appt["BookedDate"].dt, "tz", None) is not None and date_cut.tzinfo is None:
    date_cut_cmp = pd.Timestamp(date_cut, tz=df_appt["BookedDate"].dt.tz)
else:
    date_cut_cmp = pd.Timestamp(date_cut)

df_appt_filtered = df_appt[df_appt["BookedDate"] >= date_cut_cmp].copy()
save_pickle(df_appt_filtered, appt_table)

# --- 2) Derive Episode + Patient from Appointment ---
for table, id_name in [("ML_DataModel.Episode", "Episode"), ("ML_DataModel.Patient", "Patient")]:
    df1 = ORIGINAL_dfs[table].copy()
    require_cols(df1, table, ["ID"])
    require_cols(df_appt_filtered, appt_table, [id_name])

    ids = df_appt_filtered[id_name].dropna().unique()
    df1_filtered = df1[df1["ID"].isin(ids)].copy()
    save_pickle(df1_filtered, table)

# --- 3) Derive other tables from Episode ---
episode_table = "ML_DataModel.Episode"
df_episode = pd.read_pickle(SAVE_DIR / f"{episode_table}.pkl")  # or keep in memory if you prefer

require_cols(df_episode, episode_table, ["ID"])
episode_ids = df_episode["ID"].dropna().unique()

for table in ["ML_DataModel.Order", "ML_DataModel.Diagnosis", "ML_DataModel.Observation", "ML_DataModel.Procedures"]:
    df1 = ORIGINAL_dfs[table].copy()
    require_cols(df1, table, ["Episode"])

    df1_filtered = df1[df1["Episode"].isin(episode_ids)].copy()
    save_pickle(df1_filtered, table)

print(f"Saved simplified tables to: {SAVE_DIR.resolve()}")

Saved simplified tables to: C:\Users\da-jaramjo907\Documents\DS_model_dev\LoS_prediction\data\gcc_simplified\01_01_2025


# Tables Merging